<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

**Задача**

>Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
>
>**Обучите модель классифицировать комментарии на позитивные и негативные**. В вашем распоряжении набор данных с разметкой о токсичности правок.
>
>Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`.

| column | description
| ------ | :----------
| text | текст комментария
| **toxic** | **целевой признак<br>1 - токсичный комментарий<br>0 - нейтральный**

## Загрузка данных

Импортируем необходимые библиотеки.

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

nltk.download('wordnet')
nltk.download('stopwords') 

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Загрузим данные из CSV файла и сохраним их в переменной *data*. Проверим корректность загрузки данных и общую информацию.

In [2]:
data = pd.read_csv("datasets/toxic_comments.csv")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Данные загружены корректно. Столбец *text* содержит текст комментария, заметим, что строки содержат знаки форматирования. Столбец *toxic* принмает одно из числовых значений: 0 или 1.

Проверим распределение целевого признака.

In [3]:
data['toxic'].value_counts(normalize=True).round(4) * 100

0    89.83
1    10.17
Name: toxic, dtype: float64

Около 10% комментариев в выборке являются негативными.

Данные готовы к предобработке.

## Подготовка данных

Исходные данные содержат комментарии в том виде, в каком их отправляли пользователи. Тексты содержат не только слова, но и числа, знаки пунктуации и нечитаемые символы форматирования. Для определения эмоциональной окраски текста достаточно оставить только слова.

Прроведем над текстами ряд операций:
- Приведем все слова к нижнему регистру;
- Оставим только буквенные символы латинского алфавита;
- Используем лемматизатор из библиотеки ntlk.

In [4]:
lemmatizer = WordNetLemmatizer()
data_lemmas = []

for text in data['text']:
    text = text.lower()
    text = re.sub(r"[^a-z]", " ", text)

    lemmas = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    data_lemmas.append(lemmas)

In [5]:
data['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [6]:
data_lemmas[0]

'explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalism just closure on some gas after i voted at new york doll fac and please don t remove the template from the talk page since i m retired now'

Получили очищенные тексты состоящие из лемматизированных слов. Заметим, что лемматизированы были только существительные.

Составим из полученных лемм DataFrame и разделим его на обучающую и тестовую выборку в соотношении 90-10.

In [7]:
data_lemmas = pd.DataFrame(data_lemmas, columns=['text'])
data_lemmas['toxic'] = data['toxic']
train, test = train_test_split(data_lemmas, random_state=12345, train_size=0.9)

Проверим распределение целевого признака в обучающей и тестовой выборках.

In [8]:
print('Train set:\n', train['toxic'].value_counts(normalize=True))
print('\nTest set:', test['toxic'].value_counts(normalize=True))

Train set:
 0    0.898623
1    0.101377
Name: toxic, dtype: float64

Test set: 0    0.895601
1    0.104399
Name: toxic, dtype: float64


В качестве признака для предсказательной модели будем использовать **TF-IDF**. Эта величина позволит выделить слова, которыми отличаются негативные комментарии из всей выборки. Чтобы уменьшить объем выборки и повысить точность предсказаний исключим из рассмотрения предлоги, самые общеупотребительные и служебные слова.

Воспользуемся функцией *TfidfVectorizer* из библиотеки *sklearn*.

In [9]:
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(stop_words=stop_words)

tf_idf_train = vectorizer.fit_transform(train['text'])
tf_idf_test = vectorizer.transform(test['text'])

print('Sets shapes\nTrain set: {}\nTest set: {}'.format(tf_idf_train.shape, tf_idf_test.shape))

Sets shapes
Train set: (143613, 149253)
Test set: (15958, 149253)


После векторизации в наборе данных оказалось более 149 тыс. слов. Для каждого текста был составлен вектор, состоящий из величины TF-IDF каждого слова из текста.

## Обучение моделей

Для предсказания воспользуемся двумя моделями: LogisticRegression и LGBMClassifier.

Подберем параметр C для **логистической регрессии**. Мы будем использовать кросс-валидацию, для этого составим Pipiline для векторизации и обучения модели. Pipeline необходим для того, чтобы векторизация проводилась только на обучающей выборке при каждой итерации кросс-валидации. Для оценки качества модели используем метрику F1.

In [10]:
def cv_grid(model, params):
    grid = GridSearchCV(model, params, cv=3, scoring='f1', n_jobs=-1)
    grid.fit(train['text'], train['toxic'])
    
    name = model.steps[1][1].__class__.__name__
    
    print('{}\n{}\nBest score: {:.4f}\nBest param: {} - {}'.format(
        name,
        '-'*15,
        grid.best_score_,
        *list(grid.best_params_.items())[0]))
    
    return grid.best_estimator_.steps[1][1]

In [11]:
params = {'lr__C': [0.1, 1, 10]}

lr_pipe = Pipeline([('vectorizer', vectorizer),
                    ('lr', LogisticRegression(solver='saga'))])
lr_model = cv_grid(lr_pipe, params)

LogisticRegression
---------------
Best score: 0.7690
Best param: lr__C - 10


Для модели **LGBM Classifier** подберем параметр learning_rate. И получим значение метрики F1 с помощью кросс-валидации.

In [12]:
params = {'lgbm__learning_rate': [0.4, 0.5, 0.6]}

lgbm_pipe = Pipeline([('vectorizer', vectorizer),
                      ('lgbm', LGBMClassifier(n_estimators=50, max_depth=10))])
lgbm_model = cv_grid(lgbm_pipe, params)

/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


LGBMClassifier
---------------
Best score: 0.7428
Best param: lgbm__learning_rate - 0.5


Наилучшее качество предсказаний показала модель LogisticRegressor. Используем эту модель для предсказаний на тестовой выборке. Качество будем определять по метрике F1.

In [13]:
lr_model.fit(tf_idf_train, train['toxic'])
predictions = lr_model.predict(tf_idf_test)
print('Predictions on the test set\n{}\nF1: {:.4f}'.format('-'*15, f1_score(test['toxic'], predictions)))

Predictions on the test set
---------------
F1: 0.7855


Полученное значение метрики F1 больше, требуемой заказчиком 0.75.

## Выводы

**Исходные данные были очищены от лишней информации, лемматизированы и векторизованы методом выделения значений TF-IDF**. На основе преобразованных данных были обучены две предсказательные модели: LogisticRegression и LGBMClassifier. Качество предсказаний контролировалось метрикой F1 с помощью кросс-валидации. Лучшие показатели качества были получены у модели LogisticRegression, метрика F1 равна 0.769. 

Лучшая модель была использована для предсказаний на тестовой выборке данных. В результате **метрика F1 на тестовых данных равна 0.7855**. Полученное значение метрики выше, требуемой заказчиком 0.75. В результате проделанной работы была получена модель, с достаточно высокой предсказательной способностью. На основе этой модели возможна разработка инструмента для автоматической модерации комментариев пользователей. 